
Import Google Drive



In [0]:
from google.colab import drive
drive.mount('/content/drive')

Import modules

In [0]:
import os
import pickle

from os.path import isdir, join
from pathlib import Path

import librosa
import librosa.display

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, LSTM, GRU
from keras.utils import to_categorical

from sklearn.model_selection import KFold, train_test_split

Check whether Google Drive is mounted

In [0]:
!ls "/content/drive/My Drive/ASR/"

Check wav files

In [0]:
audio_path = '/content/drive/My Drive/ASR/recordings/extract/wavs/'
print(os.listdir(audio_path)[:10])